# Contact Tracer

## Introduction

Contact Tracer is a contact tracing system designed to combat the transmission of infectious diseases and pandemics. Contact Tracer is designed to assist traditional contact tracing by automatically logging an individual's contacts by detecting them via Bluetooth signals. Data is stored securely on a users device and is shared only where the user grants explicit permission. 

![Welcome](img/welcome.jpeg)

Contact Tracer is easy to use. Just sign up and leave the app on. It will automatically record nearby devices with the contact tracer installed. 

![Signup](img/signup.jpeg) ![Home](img/home.jpeg)

### Background

#### Contact Tracing
The process of identification of persons who may have come into contact with an infected person ("contacts") and subsequent collection of further information about these contacts.

#### Goals
- Interrupt ongoing transmission and reduce spread of an infection
- Alert contacts to the possibility of infection and offer preventive counseling or prophylactic care
- Offer diagnosis, counseling and treatment to already infected individuals
- Help prevent reinfection
- Learn about the epidemiology of a disease in a particular population


### Data

Due to lack of organically gathered daa, we utilize data from NYC MTA buses service. [Source](https://www.google.com)

To simulate our data, we transform their GPS logs over a 7 day period into a log of contacts. We log all instances in which two buses are located within a certain amount of distance of each other for a given time period.

#### Other Stats
- Individuals(Buses): 5719
- Days: 7
- Logs: 248900

*Disclaimer: While we do our analysis on this dataset as a proof of concept, it is unlikely that it is similar to real data.

## Code and Analysis

When a user's contacts are synced to Contact Tracer's backend, it essentially formed a network of individuals and their contacts. Here the nodes are individuals, and the edges represent who an individual has come into contact during a certain time period. With our contacts collected, we can now perform a variety of analytics. Here we demonstrate two examples: Connected Components(CC) and Ranking.

### Connected Components

For CC, we are simply interested in finding a subgraph in which any two contacts are connected to each other. Although simple, this is perhaps, the most important analysis to perform. When an infected person is identified, a CC indentifies all the individuals who may have come into contact with the infected person and who may be potentially infected themselves.

### Ranking

For Ranking, we are interested in assigning a rank or priority to individuals. A rank, in this case, can be thought of their potential to transmit a disease. As such, public health authorities should contact high ranked individuals first to mitigate the spread of a virus.

##### Import

We first start by importing some libraries.

In [ ]:
# data analysis and manipulation tool.
import pandas as pd
# scientific and vector computation for python
import numpy as np
# geocoing tool
import geopy.distance
# complex network tool
import networkx as nx
# plotting library
import matplotlib.pyplot as plt

import operator
from datetime import timedelta

# tells matplotlib to embed plots within the notebook
%matplotlib inline

# column names for our data
COLUMN_NAMES = ["datetime", "source", "target"]

##### Helper Functions

Next we define a few helper functions.

In [ ]:
def distance(lat1: str, long1: str, lat2: str, long2: str) -> float:
    """ Calculates distance between two coordinates
    Args:
        lat1 (str): latitude of first point
        long1 (str): longitude of first point
        lat2 (str): latitude of seocnd point
        long2 (str): longitude of second point
    Returns:
        float: distance between two points in feet
    """
    return geopy.distance.distance((lat1, long1), (lat2, long2)).feet


def withinDistance(row1: pd.core.series.Series, row2: pd.core.series.Series, threshold: int = 10) -> bool:
    """ Determines whether two buses were within distance of each other
    Args:
        row1 (pd.core.series.Series): record of a bus gps location
        row2 (pd.core.series.Series): record of a bus gps location
        threshold (int): distance threshold or max distance apart
    Returns:
        bool: whether buses where within distance of each other
    """
    return distance(row1.latitude, row1.longitude, row2.latitude, row2.longitude) <= threshold


def withinTime(row1: pd.core.series.Series, row2: pd.core.series.Series, threshold: int = 60) -> bool:
    """ Determines whether two buses were within a certain time period of each other
    Args:
        row1 (pd.core.series.Series): record of a bus gps location
        row2 (pd.core.series.Series): record of a bus gps location
        threshold (int): time threshold or max time apart
    Returns:
        bool: whether two buses were within a certain time period of each other
    """
    return abs((row1.datetime - row2.datetime).total_seconds()) <= threshold


def meanTimestamp(row1: pd.core.series.Series, row2: pd.core.series.Series) -> pd._libs.tslibs.timestamps.Timestamp:
    """ Gets the average of two timestamps
    Args:
        row1 (pd.core.series.Series): record of a bus gps location
        row2 (pd.core.series.Series): record of a bus gps location
    Returns:
        pd._libs.tslibs.timestamps.Timestamp: average timestamp of two bus records
    """
    return row1.datetime + (row2.datetime - row1.datetime) / 2


def sortPageRank(pr: dict, top: int = None) -> dict:
    """ Sort component based of "PageRank"
    Args:
        pr (dict): dict containing rank of each node
    Returns:
        dict: sorted dict based on rank
    """
    if not top:
        top = len(pr)
    return dict(sorted(pr.items(), key=operator.itemgetter(1), reverse=True)[:top])


def rank(pr: dict, query: set) -> dict:
    """ Get rank of connected component.
    Args:
        pr (dict): dict containing rank of each node
        query (set): set of nodes.
    Returns:
        dict: sorted nodes based on rank
    """
    results = dict((k, pr[k]) for k in query)
    return sortPageRank(results)

##### Preprocess

Next, we need a function to preprocess our data by converting it from gps coordinates to a log of contacts. The resulting data should consist of the following fields:
- datetime: when contact occured
- source: the user
- target: the users' contact

In [ ]:
def preprocess(filepath: str, days: int = 7) -> pd.core.frame.DataFrame:
    """ Preprocess data.
    Args:
        filepath (str): path of file
        days (int): number of days to process
    Returns:
        pd.core.frame.DataFrame: new preprocessed dataframe object
    """
    rename = {"RecordedAtTime": "datetime", "VehicleRef": "id",
              "VehicleLocation.Latitude": "latitude", "VehicleLocation.Longitude": "longitude"}
    headers = list(rename.keys())
    df = pd.read_csv(filepath, usecols=headers, parse_dates=[headers[0]])
    df.rename(columns=rename, inplace=True)
    df.sort_values(by=["datetime", "latitude", "longitude"],
                   inplace=True, ignore_index=True)
    df.id = df.id.astype('category').cat.codes

    start_date = df.datetime.min().normalize()
    end_date = (start_date + timedelta(days=days)).normalize()

    mask = (df['datetime'] < end_date)
    df = df.loc[mask]

    temp = []
    cnt = len(df)

    for source_idx, source_row in df.iterrows():
        for target_idx in range(source_idx + 1, cnt):
            target_row = df.loc[target_idx]
            if (withinTime(source_row, target_row) and withinDistance(source_row, target_row)):
                if (source_row.id != target_row.id):
                    temp.append([meanTimestamp(source_row, target_row),
                                 source_row.id, target_row.id])
                    temp.append([meanTimestamp(target_row, source_row),
                                 target_row.id, source_row.id])
            else:
                break

    newDF = pd.DataFrame(temp, columns=COLUMN_NAMES)
    return newDF

##### Preprocess Data

With all our functions defined, we can now load our data and proprocess it into the desired format.

In [ ]:
filepath = "mta_1706.csv"
df = preprocess(filepath)

##### Construct Graph

Having converted our dataset of gps coordinates to a log of contacts, we can now begin to analyze our data. Our log of contacts can essentially be represented as an undirected graph. Thus, the next step is to create a graph from our new dataset.

Having constructed our graph, we can now began to analyze our dataset. One way for to help us understand the data is to visualize it. Below, we create a visualization of our network. In addition, it may also be helpful to get some basic stats such as number of nodes and number of edges.

In [ ]:
G = nx.from_pandas_edgelist(df)
nx.draw_networkx(G)
plt.show()

print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")



##### Rank

Having created a graph data structure, we can now perform our first analysis, computing the rank of each individual. We use an algorithm similar to PageRank, an algorithm originally developed by Google to rank web pages.

"PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites."

One major here is that our graph is undirected. However, this can be resolved by making every undirected edge into two directed edges. 

In [ ]:
pr = sortPageRank(nx.pagerank(G, 0.4))

##### Connected Component

Next, we use a simple search algorithm to find all connected components. Now that we have both connected components and our rankings, we can now combine the two results and directly utilize them in helping us contact trace and prevent further spread of the virus. 

As an example, here we compute the rank of the largest connected component. However, in practice, when an infected individual is identified, we find the connected component that it belongs to. After identifying the connected component, we then sort then in order of their ranking which was derived from our PageRank like algorithm. Now given an infected individual, we can now easily identify those potentially infected and the best order to reach out to them. 

In [ ]:
cc = max(nx.connected_components(G), key=len)
rankedCC = rank(pr, cc)

print("rank\tid\tscore")
for i, (k, v) in enumerate(rankedCC.items(), 1):
    print("%d\t%s\t%f" %(i, k, v))

## Conclusion

Contact Tracer is a contact tracing system that consists of two main parts: a mobile app to record an individuals contacts via Bluetooth signals and a back end analytics system designed to make sense of the data and ultimately, prevent the spread of viruses and epidemics. Here we use data gathered from NYC MTA buses service to simulate real contact data. We transform it into a log of contacts, and then create a network from those logs. Having created a network, we can now perform useful analysis such as identifying Connected Components to see where a virus may have spread and Ranking, where we can quantify the importance and potential for a person to spread a disease so that we may proritize contacting those individuals. 